In [1]:
#!/usr/bin/env python
# coding: utf-8
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import PredicateMapper_utilities
from nltk.stem import WordNetLemmatizer

class PredicateMapper:
    def __init__(self, triples_path,verbs_path, embd_path):
        self.predicate_embd = PredicateMapper_utilities.load_embeddings(embd_path)
        self.verb_map = PredicateMapper_utilities.loadVerbMap(verbs_path)
        self.input_triples = PredicateMapper_utilities.read_json_file(triples_path)
        self.mapped_predicate = {}

    
    
    def similarity_mapping(self, predicate):
       
        model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        embd_predicate = model.encode(predicate, convert_to_tensor=True)
        embd_predicate = np.array(embd_predicate).reshape(1, -1)    
        max_similarity = float('-inf')
        max_element = None
        for k, v in self.predicate_embd.items():
            embd_verb = np.array(v).reshape(1, -1)
            similarity = cosine_similarity(embd_predicate, embd_verb)[0, 0]
    
            if similarity > max_similarity:
                max_similarity = similarity
                max_element = {k: similarity}
    
        # print(max_element)
        return max_element
    

    def get_last_pp(self, predicate):
        ## return a tuple the predicate with out last pp and the pp
        pass



    def check_last_word_preposition(self, predicate):
    # Liste des prépositions en anglais (à compléter si nécessaire)
        prepositions = ["aboard", "about", "above", "across", "after", "against", "along", "amid", "among", "around", 
                        "as", "at", "before", "behind", "below", "beneath", "beside", "between", "beyond", "by", 
                        "concerning", "considering", "despite", "down", "during", "except", "for", "from", "in", 
                        "inside", "into", "like", "near", "of", "off", "on", "onto", "out", "outside", "over", 
                        "past", "regarding", "round", "since", "through", "throughout", "to", "toward", "under", 
                        "underneath", "until", "unto", "up", "upon", "with", "within", "without"]
    
        words = predicate.split()
        last_word = words[-1].lower()  # Convertir en minuscules pour la comparaison en anglais
    
        if last_word in prepositions:
            # Le dernier mot est une préposition
            sentence_without_preposition = ' '.join(words[:-1])
            return (sentence_without_preposition, last_word)
        else:
            # Le dernier mot n'est pas une préposition
            return (predicate,None)


    def lemmatize_predicate(self,p):
            lemmatizer = WordNetLemmatizer()
            lemmatized_verb = lemmatizer.lemmatize(p, pos='v')
            return lemmatized_verb
    
    ## return the dic of mapped predicate
        # threshold: the predicate mapping threshold
        # option: keep or  delete for unmpapped predicate
            ## keep: keep the as them (not mention them in the dict mapping)
            ## delete: in the mapping dict add a message that indicate thist triple should be deleted ("invalid triple")
    def predicate_mapping(self, threshold, option = "keep"):
        for triple in tqdm(self.input_triples, desc="predicate mapping"):
            predicate =  self.lemmatize_predicate(triple["predicate"])
            predicate_ = self.check_last_word_preposition(predicate)
            if predicate_:
                if predicate_[0] in self.verb_map.keys(): 
                    pp = ""
                    if predicate_[1]:
                        pp = " "+ predicate_[1]
                    self.mapped_predicate[predicate] = self.lemmatize_predicate(self.verb_map[predicate_[0]]) + pp
                    # print("direct mapping:  ", predicate,"-->" ,self.verb_map[predicate_[0]] + pp )
                else:
                   
                    max_sim_verb = self.similarity_mapping(predicate_[0])
                    # print(max_sim_verb)
                    if list(max_sim_verb.values())[0] >= threshold:
                        pp = ""
                        if predicate_[1]:
                            pp = " "+ predicate_[1]
                        self.mapped_predicate[predicate] =  self.lemmatize_predicate(self.verb_map[list(max_sim_verb.keys())[0]]) + pp
                        # print("sim:",max_sim_verb,"\n")
                        # print("similarity mapping:  ", predicate,"-->", self.verb_map[list(max_sim_verb.keys())[0]] + pp )
                    else:
                        ### keep or delete ! 
                        if option == "delete":
                            self.mapped_predicate[predicate] = "invalid triple"
                        if option == "keep":
                            pass
                        else:
                            print("invalid option !")
                            break       

C:\Users\admin-user\anaconda3\envs\mapping-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def lemmatize_predicate(p):
            lemmatizer = WordNetLemmatizer()
            lemmatized_verb = lemmatizer.lemmatize(p, pos='v')
            return lemmatized_verb

'affect'